# Inserting the Data

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Add, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from itertools import product

# Verify GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

base_dir = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\data_aug_3'
subfolders = ['clear', 'clouds']
categories = ['Healthy_augmented', 'Damaged_augmented']
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 8

def load_data(base_dir, subfolders, categories, img_height, img_width):
    data = []
    labels = []
    for category in categories:
        class_num = categories.index(category)
        clear_path = os.path.join(base_dir, subfolders[0], category)
        clouds_path = os.path.join(base_dir, subfolders[1], category)
        clear_images = sorted(os.listdir(clear_path))
        clouds_images = sorted(os.listdir(clouds_path))
        
        for clear_img_name, clouds_img_name in zip(clear_images, clouds_images):
            if clear_img_name.endswith('.png') and clouds_img_name.endswith('.png'):
                clear_img_path = os.path.join(clear_path, clear_img_name)
                clouds_img_path = os.path.join(clouds_path, clouds_img_name)
                
                clear_img = tf.keras.preprocessing.image.load_img(clear_img_path, target_size=(img_height, img_width))
                clouds_img = tf.keras.preprocessing.image.load_img(clouds_img_path, target_size=(img_height, img_width))
                
                clear_img_array = tf.keras.preprocessing.image.img_to_array(clear_img)
                clouds_img_array = tf.keras.preprocessing.image.img_to_array(clouds_img)
                
                combined_img = np.concatenate((clear_img_array, clouds_img_array), axis=-1)
                
                data.append(combined_img)
                labels.append(class_num)
    return np.array(data), np.array(labels)

data, labels = load_data(base_dir, subfolders, categories, IMG_HEIGHT, IMG_WIDTH)

# Normalize the images
data = data / 255.0

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation labels shape: {y_val.shape}")


TensorFlow version: 2.10.0
Num GPUs Available:  1
Training data shape: (4832, 64, 64, 6)
Validation data shape: (1208, 64, 64, 6)
Training labels shape: (4832, 2)
Validation labels shape: (1208, 2)
Class weights: {0: 0.8640915593705293, 1: 1.18664047151277}

Initial Training Combination 1/50: num_residual_blocks=4, dropout_rate=0.3, learning_rate=0.001, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.3, zoom_range=0.1, horizontal_flip=True


C:\Users\NickZografos\anaconda3\envs\thesis\lib\site-packages\keras\preprocessing\image.py:2094: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (4832, 64, 64, 6) (6 channels).
  warnings.warn(
C:\Users\NickZografos\anaconda3\envs\thesis\lib\site-packages\keras\preprocessing\image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (4832, 64, 64, 6) (6 channels).
  warnings.warn(


Epoch 1/40
604/604 [==============================] - 18s 19ms/step - loss: 0.7710 - accuracy: 0.6215 - val_loss: 1.6062 - val_accuracy: 0.4007 - lr: 0.0010
Epoch 2/40
604/604 [==============================] - 11s 18ms/step - loss: 0.6081 - accuracy: 0.6892 - val_loss: 0.6777 - val_accuracy: 0.6846 - lr: 0.0010
Epoch 3/40
604/604 [==============================] - 11s 18ms/step - loss: 0.5818 - accuracy: 0.7067 - val_loss: 0.7988 - val_accuracy: 0.4669 - lr: 0.0010
Epoch 4/40
604/604 [==============================] - 11s 18ms/step - loss: 0.5694 - accuracy: 0.7235 - val_loss: 0.6836 - val_accuracy: 0.5877 - lr: 0.0010
Epoch 5/40
604/604 [==============================] - 11s 18ms/step - loss: 0.5682 - accuracy: 0.7276 - val_loss: 0.5165 - val_accuracy: 0.7334 - lr: 0.0010
Epoch 6/40
604/604 [==============================] - 11s 18ms/step - loss: 0.5644 - accuracy: 0.7146 - val_loss: 4.1128 - val_accuracy: 0.6026 - lr: 0.0010
Epoch 7/40
604/604 [==============================] - 11s 

# Finding the best combination of parameters

In [ ]:
# Define data augmentation
def create_datagen(rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip):
    return ImageDataGenerator(
        rotation_range=rotation_range,
        width_shift_range=width_shift_range,
        height_shift_range=height_shift_range,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip,
        fill_mode='nearest'
    )

def residual_block(x, filters):
    shortcut = x
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)
        
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def build_complex_model(input_shape, num_residual_blocks, dropout_rate, learning_rate):
    # Input layer for combined images
    combined_input = Input(shape=(input_shape[1], input_shape[2], input_shape[3]), name='combined_input')
    
    # Convolutional base
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(combined_input)
    x = MaxPooling2D((2, 2))(x)
    for _ in range(num_residual_blocks):
        x = residual_block(x, 64)
        x = MaxPooling2D((2, 2))(x)
    x = GlobalAveragePooling2D()(x)
    
    # Fully connected layers
    x = Dense(2048, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(2, activation='softmax')(x)

    model = Model(inputs=combined_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = X_train.shape

# Grid search parameters
num_residual_blocks_options = [1, 2, 3, 4, 5]
dropout_rate_options = [0.3, 0.4, 0.5, 0.6]
learning_rate_options = [1e-2, 1e-3, 5e-4, 1e-4]
rotation_range_options = [10, 20, 30]
width_shift_range_options = [0.1, 0.2, 0.3]
height_shift_range_options = [0.1, 0.2, 0.3]
shear_range_options = [0.1, 0.2, 0.3]
zoom_range_options = [0.1, 0.2, 0.3]
horizontal_flip_options = [True, False]

# Create grid search parameter combinations
parameter_combinations = list(product(num_residual_blocks_options, dropout_rate_options, learning_rate_options,
                                      rotation_range_options, width_shift_range_options, height_shift_range_options,
                                      shear_range_options, zoom_range_options, horizontal_flip_options))

# Select 50 unique combinations
np.random.seed(42)
selected_combinations = np.random.choice(len(parameter_combinations), 50, replace=False)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

print(f"Class weights: {class_weights}")

# Callbacks for training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# Custom callback to print epoch details
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}/{self.params['epochs']}: loss={logs['loss']:.4f}, accuracy={logs['accuracy']:.4f}, val_loss={logs['val_loss']:.4f}, val_accuracy={logs['val_accuracy']:.4f}")

# Perform grid search
best_val_accuracy = 0
best_params = None

for idx, combination_idx in enumerate(selected_combinations):
    num_residual_blocks, dropout_rate, learning_rate, rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip = parameter_combinations[combination_idx]
    print(f"\nInitial Training Combination {idx + 1}/50: num_residual_blocks={num_residual_blocks}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, rotation_range={rotation_range}, width_shift_range={width_shift_range}, height_shift_range={height_shift_range}, shear_range={shear_range}, zoom_range={zoom_range}, horizontal_flip={horizontal_flip}")
    
    datagen = create_datagen(rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip)
    datagen.fit(X_train)
    train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
    
    model = build_complex_model(input_shape, num_residual_blocks, dropout_rate, learning_rate)
    
    with tf.device('/GPU:0'):
        history = model.fit(
            train_generator,
            steps_per_epoch=len(X_train) // BATCH_SIZE,
            epochs=40,
            validation_data=(X_val, y_val),
            callbacks=[reduce_lr, early_stopping, CustomCallback()],
            class_weight=class_weights,
            verbose=1
        )
    
    val_accuracy = max(history.history['val_accuracy'])
    print(f"Validation accuracy: {val_accuracy}")
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_params = (num_residual_blocks, dropout_rate, learning_rate, rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip)

print(f"\nBest parameters found in initial grid search: num_residual_blocks={best_params[0]}, dropout_rate={best_params[1]}, learning_rate={best_params[2]}, rotation_range={best_params[3]}, width_shift_range={best_params[4]}, height_shift_range={best_params[5]}, shear_range={best_params[6]}, zoom_range={best_params[7]}, horizontal_flip={best_params[8]}")
print(f"Best validation accuracy: {best_val_accuracy}")

# Refine grid search around best parameters
refined_num_residual_blocks_options = [best_params[0] - 1, best_params[0], best_params[0] + 1]
refined_dropout_rate_options = [max(0.0, best_params[1] - 0.1), best_params[1], min(1.0, best_params[1] + 0.1)]
refined_learning_rate_options = [best_params[2] * 0.5, best_params[2], best_params[2] * 2]
refined_rotation_range_options = [max(0, best_params[3] - 10), best_params[3], best_params[3] + 10]
refined_width_shift_range_options = [max(0.0, best_params[4] - 0.1), best_params[4], min(1.0, best_params[4] + 0.1)]
refined_height_shift_range_options = [max(0.0, best_params[5] - 0.1), best_params[5], min(1.0, best_params[5] + 0.1)]
refined_shear_range_options = [max(0.0, best_params[6] - 0.1), best_params[6], min(1.0, best_params[6] + 0.1)]
refined_zoom_range_options = [max(0.0, best_params[7] - 0.1), best_params[7], min(1.0, best_params[7] + 0.1)]
refined_horizontal_flip_options = [best_params[8]]

refined_parameter_combinations = list(product(refined_num_residual_blocks_options, refined_dropout_rate_options, refined_learning_rate_options,
                                              refined_rotation_range_options, refined_width_shift_range_options, refined_height_shift_range_options,
                                              refined_shear_range_options, refined_zoom_range_options, refined_horizontal_flip_options))

# Select 50 unique combinations for refined search
np.random.seed(42)
selected_refined_combinations = np.random.choice(len(refined_parameter_combinations), 50, replace=False)

# Perform refined grid search
best_val_accuracy_refined = 0
best_params_refined = None

for idx, combination_idx in enumerate(selected_refined_combinations):
    num_residual_blocks, dropout_rate, learning_rate, rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip = refined_parameter_combinations[combination_idx]
    print(f"\nRefined Training Combination {idx + 1}/50: num_residual_blocks={num_residual_blocks}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, rotation_range={rotation_range}, width_shift_range={width_shift_range}, height_shift_range={height_shift_range}, shear_range={shear_range}, zoom_range={zoom_range}, horizontal_flip={horizontal_flip}")
    
    datagen = create_datagen(rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip)
    datagen.fit(X_train)
    train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
    
    model = build_complex_model(input_shape, num_residual_blocks, dropout_rate, learning_rate)
    
    with tf.device('/GPU:0'):
        history = model.fit(
            train_generator,
            steps_per_epoch=len(X_train) // BATCH_SIZE,
            epochs=40,
            validation_data=(X_val, y_val),
            callbacks=[reduce_lr, early_stopping, CustomCallback()],
            class_weight=class_weights,
            verbose=1
        )
    
    val_accuracy = max(history.history['val_accuracy'])
    print(f"Validation accuracy: {val_accuracy}")
    
    if val_accuracy > best_val_accuracy_refined:
        best_val_accuracy_refined = val_accuracy
        best_params_refined = (num_residual_blocks, dropout_rate, learning_rate, rotation_range, width_shift_range, height_shift_range, shear_range, zoom_range, horizontal_flip)

print(f"\nRefined best parameters found: num_residual_blocks={best_params_refined[0]}, dropout_rate={best_params_refined[1]}, learning_rate={best_params_refined[2]}, rotation_range={best_params_refined[3]}, width_shift_range={best_params_refined[4]}, height_shift_range={best_params_refined[5]}, shear_range={best_params_refined[6]}, zoom_range={best_params_refined[7]}, horizontal_flip={best_params_refined[8]}")
print(f"Refined best validation accuracy: {best_val_accuracy_refined}")

# Train the final model with the refined best parameters found
final_model = build_complex_model(input_shape, best_params_refined[0], best_params_refined[1], best_params_refined[2])

with tf.device('/GPU:0'):
    final_history = final_model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=50,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr, early_stopping, CustomCallback()],
        class_weight=class_weights,
        verbose=1
    )

# Make predictions
with tf.device('/GPU:0'):
    val_predictions = final_model.predict(X_val)

# Convert one-hot encoded predictions and true labels to label indices
y_val_true = np.argmax(y_val, axis=1)
y_val_pred = np.argmax(val_predictions, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_val_true, y_val_pred)

print("Confusion Matrix:")
print(conf_matrix)

# Generate the classification report
class_report = classification_report(y_val_true, y_val_pred, target_names=categories)

print("Classification Report:")
print(class_report)

In [6]:
#best_params_refined = (5, 0.19999999999999998, 0.001, 40, 0.1, 0.1, 0.2, 0.2, True)

In [7]:
best_params_refined

(5, 0.19999999999999998, 0.001, 40, 0.1, 0.1, 0.2, 0.2, True)

In [3]:
best_val_accuracy_refined

0.8741722106933594

# Re-training the best model (NOT REQUIRED)

In [8]:
best_params_refined = (5, 0.19999999999999998, 0.001, 40, 0.1, 0.1, 0.2, 0.2, True)
datagen = create_datagen(best_params_refined[3], best_params_refined[4], best_params_refined[5], best_params_refined[6], best_params_refined[7], best_params_refined[8])
datagen.fit(X_train)
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

# Train the final model with the refined best parameters found
final_model = build_complex_model(input_shape, best_params_refined[0], best_params_refined[1], best_params_refined[2])

with tf.device('/GPU:0'):
    final_history = final_model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=50,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr, early_stopping, CustomCallback()],
        class_weight=class_weights,
        verbose=1
    )

# Make predictions
with tf.device('/GPU:0'):
    val_predictions = final_model.predict(X_val)

# Convert one-hot encoded predictions and true labels to label indices
y_val_true = np.argmax(y_val, axis=1)
y_val_pred = np.argmax(val_predictions, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_val_true, y_val_pred)

print("Confusion Matrix:")
print(conf_matrix)

# Generate the classification report
class_report = classification_report(y_val_true, y_val_pred, target_names=categories)

print("Classification Report:")
print(class_report)

C:\Users\NickZografos\anaconda3\envs\thesis\lib\site-packages\keras\preprocessing\image.py:2094: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (4832, 64, 64, 6) (6 channels).
  warnings.warn(
C:\Users\NickZografos\anaconda3\envs\thesis\lib\site-packages\keras\preprocessing\image.py:766: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (4832, 64, 64, 6) (6 channels).
  warnings.warn(


Epoch 1/50
604/604 [==============================] - 14s 20ms/step - loss: 0.7987 - accuracy: 0.6026 - val_loss: 0.8140 - val_accuracy: 0.6581 - lr: 0.0010
Epoch 2/50
604/604 [==============================] - 12s 19ms/step - loss: 0.5945 - accuracy: 0.7059 - val_loss: 0.7182 - val_accuracy: 0.6010 - lr: 0.0010
Epoch 3/50
604/604 [==============================] - 12s 19ms/step - loss: 0.5505 - accuracy: 0.7363 - val_loss: 0.5268 - val_accuracy: 0.7334 - lr: 0.0010
Epoch 4/50
604/604 [==============================] - 12s 19ms/step - loss: 0.5537 - accuracy: 0.7355 - val_loss: 0.5170 - val_accuracy: 0.7732 - lr: 0.0010
Epoch 5/50
604/604 [==============================] - 12s 19ms/step - loss: 0.5472 - accuracy: 0.7457 - val_loss: 0.5730 - val_accuracy: 0.7028 - lr: 0.0010
Epoch 6/50
604/604 [==============================] - 12s 19ms/step - loss: 0.5432 - accuracy: 0.7363 - val_loss: 0.6746 - val_accuracy: 0.6225 - lr: 0.0010
Epoch 7/50
604/604 [==============================] - 12s 

# Save the Best Model 

In [10]:
# Save the best model to a local directory
#model_save_path = 'best_model.h5'
#final_model.save(model_save_path)
#print(f"Best model saved to {model_save_path}")

Best model saved to best_model.h5


# Load the Best Model

In [ ]:
from tensorflow.keras.models import load_model

# Load the best model from the local directory
model_save_path = 'best_model.h5'
loaded_model = load_model(model_save_path)
print(f"Model loaded from {model_save_path}")

# Make predictions using the loaded model
with tf.device('/GPU:0'):
    loaded_val_predictions = loaded_model.predict(X_val)

# Convert one-hot encoded predictions and true labels to label indices
loaded_y_val_pred = np.argmax(loaded_val_predictions, axis=1)

# Generate the confusion matrix for the loaded model
loaded_conf_matrix = confusion_matrix(y_val_true, loaded_y_val_pred)

print("Confusion Matrix for loaded model:")
print(loaded_conf_matrix)

# Generate the classification report for the loaded model
loaded_class_report = classification_report(y_val_true, loaded_y_val_pred, target_names=categories)

print("Classification Report for loaded model:")
print(loaded_class_report)